In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import requests
from spotipy.oauth2 import SpotifyOAuth
from tqdm import tqdm
import sys
import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
import time
from google.cloud import bigquery
import os

import pandas_gbq
%load_ext google.cloud.bigquery

# Set your default project here
pandas_gbq.context.project = 'snappy-boulder-378707'
pandas_gbq.context.dialect = 'standard'
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "../letsgo-snappy-boulder-378707-4b7d46801fd1.json"

# Construct a BigQuery client object.
client = bigquery.Client()


from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler, RobustScaler, OneHotEncoder

# Genre

In [2]:
import joblib
import pickle
gb = pickle.load(open('../../Model/gb_0.4343.sav','rb'))
#gb = joblib.load('../../Model/gb_0.4343.sav')

In [156]:
%%bigquery genre
SELECT distinct * FROM snappy-boulder-378707.NewReleases.NewAudioFeatures

Query is running:   0%|          |

Downloading:   0%|          |

In [157]:
dummy_cols = ['key','mode','time_signature']
genre= pd.get_dummies(genre, columns=dummy_cols)
genre.drop(['energy', 'loudness'],axis=1, inplace=True)

In [158]:
features = pd.read_csv("../../Model/Genre_classification_features.csv")

In [159]:
for col in set(features.iloc[:,0].values) - set(genre.iloc[:,1:].columns):
    genre[col] = 0
sc = MinMaxScaler(feature_range = (0,100))
genre_trans = sc.fit_transform(genre.iloc[:,1:])

In [160]:
pred_genre = gb.predict(genre_trans)

In [162]:
genre['Genre'] = pred_genre

In [164]:
genre = genre[['id','Genre']]

In [165]:
genre

,id,Genre
0,7rXQXbK4pQo8DSKlZ9ZSjC,K-Pop
1,2LRqm962By8aNwJX0WxFNG,Folk & Acoustic
2,0Oqxt6JixieLHbwMfnJGWO,Jazz
3,0tYtfBFJLVn2ITE66FbaLT,Indie
4,5ZSqX1dDJ823Dndw94nogY,Dance Electronic
...,...,...
179,1Oi8jp3BvmESVXIPtJ06a5,K-Pop
180,2lxwvwNTCT6T2k9agnujeO,Mandopop
181,6f6jhTzxwvAYhzZrABTO8T,Malay Pop
182,4Fx7GtXOK8k3rxVgKzi6ko,Mandopop


# Popularity

In [88]:
%%bigquery popularity
SELECT distinct s1.id,
s1.explicit,
s1.available_markets,
artist_id,
danceability,
energy,
key,
loudness,
mode,
speechiness,
acousticness,
instrumentalness,
liveness,
valence,
tempo,
duration_ms,
time_signature,
release_date FROM `snappy-boulder-378707.NewReleases.NewTracks`  as s1
inner join `snappy-boulder-378707.NewReleases.NewAudioFeatures`as s2
on s1.id = s2.id
inner join `snappy-boulder-378707.NewReleases.NewAlbums` as s3
on s1.album_id = s3.id

Query is running:   0%|          |

Downloading:   0%|          |

In [89]:
%%bigquery artist
SELECT distinct id, followers, popularity FROM `snappy-boulder-378707.TrackClearInfo.ArtistInfo`

Query is running:   0%|          |

Downloading:   0%|          |

In [90]:
popularity = popularity.assign(artist_id=popularity.artist_id.str.split(";")).explode('artist_id')

In [91]:
popularity = pd.merge(popularity, artist, left_on="artist_id",right_on="id", how="left")

In [92]:
popularity.columns = ['id_track', 'explicit', 'available_markets', 'artist_id', 'danceability',
       'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms',
       'time_signature', 'release_date', 'artist_id', 'followers', 'popularity_artist']
popularity.drop_duplicates(inplace=True)

In [93]:
popularity = popularity.groupby(['id_track', 'explicit', 'available_markets','danceability',
       'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms',
       'time_signature', 'release_date'], as_index=False).aggregate({'followers':'mean','popularity_artist':'mean'})

In [94]:
import datetime as dt
popularity['release_date']=popularity['release_date'].map(dt.datetime.toordinal)
prediction = popularity[['id_track']]
popularity.drop(['id_track'],axis=1, inplace=True)

In [95]:
popularity['instrumentalness'] = popularity['instrumentalness'].apply(lambda x: 1 if x < 0.1 else (3 if x > 0.4 else 2))
popularity['speechiness'] = popularity['speechiness'].apply(lambda x: 1 if x < 0.1 else (3 if x > 0.95 else 2))

In [96]:
ct = joblib.load('../../Model/column_transfomer.joblib')

In [97]:
# ct = ColumnTransformer([('minmax', MinMaxScaler(), ['release_date','loudness', 'tempo', 'duration_ms','available_markets','followers','popularity_artist']),
#                         ('categorical', OneHotEncoder(), ['key','mode','time_signature','explicit','instrumentalness','speechiness']),
#                        ],
#                        remainder='passthrough')

popularity = pd.DataFrame(ct.transform(popularity), columns=ct.get_feature_names_out())

In [98]:
popularity_features = pd.read_csv("../../Model/Popularity_features.csv")

In [99]:
set(popularity_features.iloc[:,0].values) - set(popularity.columns)

set()

In [100]:
for col in set(popularity_features.iloc[:,0].values) - set(popularity.columns):
    popularity[col] = 0

In [101]:
import joblib
lr = joblib.load(open('../../Model/Popularity Prediction Model.sav','rb'))

In [107]:
pred_lr = lr.predict(popularity)
prediction['popularity'] = pred_lr